In [ ]:
#da eseguire solo la prima volta per installare la libreria
!pip install pathlib

In [ ]:
#libraries
from geopy.geocoders import Yandex
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date as dt
from tqdm import tqdm
from pathlib import Path
import time

big_iphone = 0
i = 700
pathlist = Path('').glob('**/*gps.csv')

for path in pathlist:
    
    i += 1
    
    #import files
    file = str(path)
    print(file, i)
    h = pd.read_csv(file, sep = "\t", skiprows=1) #gps
    r = pd.read_csv(str(i)+'_sj.csv') #science_journal
    
    #adjust gps file
    h = h.filter(["Data","Latitudine", "Longitudine", "Altitudine (m)","Velocità (km/h)"])
    h.columns = ["Time","Latitude","Longitude","Elevation","Speed"]
    h = h.apply(lambda x: x.str.replace(",","."))
    h['Time']= pd.to_datetime(h['Time'], format='%Y-%m-%d %H:%M:%S.%f') 
    h["Time"] = h['Time'].apply(lambda x: x.replace(microsecond=0))
    h[["Latitude","Longitude","Elevation","Speed"]] = h[["Latitude","Longitude","Elevation","Speed"]].apply(pd.to_numeric)
    
    #put n.person car type columns
    h['N_person'] = int(file.split('_')[2])
    h['Car_type'] = int(file.split('_')[3])
    
    #create Diff column
    data1=h.iloc[0]['Time']
    h['Diff']=(h['Time']-data1).dt.seconds
    h['Diff']=h['Diff']*1000
    
    #assign a reference to sj_file
    l=[str(i) for i in h.index]
    gap=[h.iloc[i]['Diff'] for i in range(len(h))]
    del l[-1]
    r=r.assign(tag=pd.cut(r['relative_time'],gap,labels=l))
    r['tag']=r['tag'].fillna(value='0')
    
    #adjust sj
    sj_new=r.filter(['relative_time','AccY','tag'])
    sj_new.columns=['Time','Acc','Reference']
    sj_new=sj_new.astype({'Reference':'int'})
    
    #merged df
    df=pd.merge(sj_new,h,left_on='Reference',right_on=h.index )
    df=df.drop(columns=['Reference','Diff'])
    df.columns=['Elapse(ms)','Acc','Date','Latitude','Longitude','Elevation','Speed(Km/h)','N_person','Car_type']
    df=df.reindex(['Date','Elapse(ms)','Latitude','Longitude','Elevation','Speed(Km/h)','Acc','N_person','Car_type'],axis=1)              
    df=df[df['Speed(Km/h)']>1]
    
    #reverse geopy
    #ALE INSERISCI CODICE QUI!
    
    #append current df to previuos ones
    if big_iphone == 0:
        big_iphone = df
    else:
        big_iphone = big_iphone.append(df) 

big_iphone.to_csv('All_Records_iPhone.csv', index=False)